In [1]:
import cv2
import math
import numpy as np
import scipy.ndimage

In [2]:
def orientated_non_max_suppression(mag, ang):
    ang_quant = np.round(ang / (np.pi/4)) % 4
    winE = np.array([[0, 0, 0],[1, 1, 1], [0, 0, 0]])
    winSE = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    winS = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]])
    winSW = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

    magE = non_max_suppression(mag, winE)
    magSE = non_max_suppression(mag, winSE)
    magS = non_max_suppression(mag, winS)
    magSW = non_max_suppression(mag, winSW)

    mag[ang_quant == 0] = magE[ang_quant == 0]
    mag[ang_quant == 1] = magSE[ang_quant == 1]
    mag[ang_quant == 2] = magS[ang_quant == 2]
    mag[ang_quant == 3] = magSW[ang_quant == 3]
    return mag


In [3]:
def non_max_suppression(data, win):
    data_max = scipy.ndimage.filters.maximum_filter(data, footprint=win, mode='constant')
    data_max[data != data_max] = 0
    return data_max


In [4]:
# start calulcation
gray_image = cv2.imread(r'/Users/arushigupta/Desktop/Intership/Tires/Tire Data/Training/Cracked-19.jpg', 0)

with_nmsup = True #apply non-maximal suppression
fudgefactor = 1.3 
sigma = 21 #for Gaussian Kernel
kernel = 2*math.ceil(2*sigma)+1 #Kernel size

gray_image = gray_image/255.0
blur = cv2.GaussianBlur(gray_image, (kernel, kernel), sigma)
gray_image = cv2.subtract(gray_image, blur)

In [5]:
#sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
#sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
#mag = np.hypot(sobelx, sobely)
#ang = np.arctan2(sobely, sobelx)

In [6]:
scharrx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
scharry = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
mag = np.hypot(scharrx, scharry)
ang = np.arctan2(scharry, scharrx)

In [7]:
# threshold
threshold = 4 * fudgefactor * np.mean(mag)
mag[mag < threshold] = 0

In [8]:
#either get edges directly
if with_nmsup is False:
    mag = cv2.normalize(mag, 0, 255, cv2.NORM_MINMAX)
    kernel = np.ones((5,5),np.uint8)
    result = cv2.morphologyEx(mag, cv2.MORPH_CLOSE, kernel)
    cv2.imshow('im', result)
    cv2.waitKey()

#or apply a non-maximal suppression
else:

    # non-maximal suppression
    mag = orientated_non_max_suppression(mag, ang)
    # create mask
    mag[mag > 0] = 255
    mag = mag.astype(np.uint8)

    kernel = np.ones((5,5),np.uint8)
    result = cv2.morphologyEx(mag, cv2.MORPH_CLOSE, kernel)

    cv2.imshow('im', result)
    cv2.waitKey(0)
    

/var/folders/py/xhjr5n9s729_02xr2vxm8km80000gn/T/ipykernel_45045/2386538402.py:2: DeprecationWarning: Please use `maximum_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  data_max = scipy.ndimage.filters.maximum_filter(data, footprint=win, mode='constant')


In [9]:
cv2.destroyWindow("shown_img")